# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [15]:
url = 'https://scanner.tradingview.com/canada/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["canada"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","Perf.W","Perf.1M","Perf.3M","Perf.6M","Perf.Y","average_volume_90d_calc","EMA30","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"name","sortOrder":"asc"},"price_conversion":{"to_symbol":False},"range":[0,3545]}
data = requests.post(url, json = payload)

In [16]:
stocks_data = data.json()['data']
len(stocks_data)

3545

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [17]:
stocks_data[0]

{'s': 'TSX:AAB',
 'd': ['',
  'AAB',
  0.04,
  -11.11111111,
  -27.27272727,
  -20,
  -42.85714286,
  -68,
  89634.5,
  0.04697478,
  'ABERDEEN INTERNATIONAL INC',
  'stock',
  'common',
  'delayed_streaming_900',
  1000,
  5,
  'false',
  0,
  'CAD',
  'CAD']}

In [18]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][10]
    stock['Price'] = data['d'][2]
    stock['weeklyPerformance'] = data['d'][3]
    stock['1monthPerformace'] = data['d'][4]
    stock['3monthPerformace'] = data['d'][5]
    stock['6monthPerformace'] = data['d'][6]
    stock['1yearPerformace'] = data['d'][7]
    stock['AvgVolume'] = data['d'][8]
    stock['Exp Moving Average'] = data['d'][9]
    
    stats_from_tradingview.append(stock)

In [19]:
stats_from_tradingview[0]

{'Symbol': 'AAB',
 'Name': 'ABERDEEN INTERNATIONAL INC',
 'Price': 0.04,
 'weeklyPerformance': -11.11111111,
 '1monthPerformace': -27.27272727,
 '3monthPerformace': -20,
 '6monthPerformace': -42.85714286,
 '1yearPerformace': -68,
 'AvgVolume': 89634.5,
 'Exp Moving Average': 0.04697478}

In [20]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("canada_stocks_all_performance.json", "w") as outfile:
    outfile.write(json_object)

In [21]:
hqm_dataframe = pd.read_json('canada_stocks_all_performance.json')

In [22]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
hqm_dataframe[hqm_dataframe['1yearPerformace'].isna()]

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average


In [23]:
hqm_dataframe['Exp Moving Average'].fillna(hqm_dataframe['Price']+1, inplace=True)


In [24]:
hqm_dataframe['Diff From Moving Avg'] = hqm_dataframe['Exp Moving Average'] - hqm_dataframe['Price']
hqm_dataframe['Diff From Moving Avg'] = hqm_dataframe['Diff From Moving Avg'].apply(abs)

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [25]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

In [26]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg
0,AAB,ABERDEEN INTERNATIONAL INC,0.040,-11.111111,-27.272727,-20.000000,-42.857143,-68.000000,8.963450e+04,0.046975,0.006975
1,AAG,AFTERMATH SILVER LTD,0.255,13.333333,-8.928571,-23.880597,50.000000,-32.000000,1.644119e+05,0.250693,0.004307
2,AALI,ADVANCE LITHIUM CORP,0.060,71.428571,140.000000,200.000000,300.000000,9.090909,4.760062e+05,0.033653,0.026347
3,AAU,ANGOLD RESOURCES LTD,0.025,25.000000,0.000000,-28.571429,-70.588235,-85.294118,3.024310e+05,0.024456,0.000544
4,AAV,ADVANTAGE ENERGY LTD,8.380,-2.558140,4.358655,-24.910394,-20.266413,10.993377,1.008012e+06,8.383976,0.003976
...,...,...,...,...,...,...,...,...,...,...,...
3540,ZWT,BMO COVERED CALL TECHNOLOGY ETF UNIT,26.610,3.179527,-0.708955,7.298387,6.567881,-10.404040,4.597122e+03,25.833127,0.776873
3541,ZWU,BMO ASSET MANAGEMENT INC COVERED CALL UTILITIE...,11.330,-0.176211,-2.746781,-3.983051,-9.504792,-12.644564,2.707973e+05,11.446945,0.116945
3542,ZXM,CI MORNINGSTAR INTL MOMENTUM IDX ET HEDGED COM...,35.360,1.902017,3.452311,5.269425,2.492754,-0.394366,1.304700e+03,34.335796,1.024204
3543,ZZZ,SLEEP COUNTRY CANADA HOLDINGS INC,25.350,2.053140,0.000000,10.940919,-2.537486,-17.292007,9.888503e+04,24.781762,0.568238


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [27]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg
0,AAB,ABERDEEN INTERNATIONAL INC,0.040,-11.111111,-27.272727,-20.000000,-42.857143,-68.000000,8.963450e+04,0.046975,0.006975
1,AAG,AFTERMATH SILVER LTD,0.255,13.333333,-8.928571,-23.880597,50.000000,-32.000000,1.644119e+05,0.250693,0.004307
2,AALI,ADVANCE LITHIUM CORP,0.060,71.428571,140.000000,200.000000,300.000000,9.090909,4.760062e+05,0.033653,0.026347
3,AAU,ANGOLD RESOURCES LTD,0.025,25.000000,0.000000,-28.571429,-70.588235,-85.294118,3.024310e+05,0.024456,0.000544
4,AAV,ADVANTAGE ENERGY LTD,8.380,-2.558140,4.358655,-24.910394,-20.266413,10.993377,1.008012e+06,8.383976,0.003976
...,...,...,...,...,...,...,...,...,...,...,...
3540,ZWT,BMO COVERED CALL TECHNOLOGY ETF UNIT,26.610,3.179527,-0.708955,7.298387,6.567881,-10.404040,4.597122e+03,25.833127,0.776873
3541,ZWU,BMO ASSET MANAGEMENT INC COVERED CALL UTILITIE...,11.330,-0.176211,-2.746781,-3.983051,-9.504792,-12.644564,2.707973e+05,11.446945,0.116945
3542,ZXM,CI MORNINGSTAR INTL MOMENTUM IDX ET HEDGED COM...,35.360,1.902017,3.452311,5.269425,2.492754,-0.394366,1.304700e+03,34.335796,1.024204
3543,ZZZ,SLEEP COUNTRY CANADA HOLDINGS INC,25.350,2.053140,0.000000,10.940919,-2.537486,-17.292007,9.888503e+04,24.781762,0.568238


In [28]:
time_periods = [
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'Diff From Moving Avg'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile
0,AAB,ABERDEEN INTERNATIONAL INC,0.040,-11.111111,-27.272727,-20.000000,-42.857143,-68.000000,8.963450e+04,0.046975,0.006975,7.489422,4.781382,12.440056,10.014104,10.592384,19.830748
1,AAG,AFTERMATH SILVER LTD,0.255,13.333333,-8.928571,-23.880597,50.000000,-32.000000,1.644119e+05,0.250693,0.004307,90.141044,23.413258,10.380818,91.283498,34.414669,14.386460
2,AALI,ADVANCE LITHIUM CORP,0.060,71.428571,140.000000,200.000000,300.000000,9.090909,4.760062e+05,0.033653,0.026347,99.181946,99.520451,98.970381,99.407616,85.923836,41.128350
3,AAU,ANGOLD RESOURCES LTD,0.025,25.000000,0.000000,-28.571429,-70.588235,-85.294118,3.024310e+05,0.024456,0.000544,95.345557,62.496474,8.011283,1.678420,3.018336,2.397743
4,AAV,ADVANTAGE ENERGY LTD,8.380,-2.558140,4.358655,-24.910394,-20.266413,10.993377,1.008012e+06,8.383976,0.003976,21.636107,79.407616,9.901269,23.159379,87.277856,13.370945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,ZWT,BMO COVERED CALL TECHNOLOGY ETF UNIT,26.610,3.179527,-0.708955,7.298387,6.567881,-10.404040,4.597122e+03,25.833127,0.776873,72.863188,54.442877,68.434415,64.598025,53.991537,90.070522
3541,ZWU,BMO ASSET MANAGEMENT INC COVERED CALL UTILITIE...,11.330,-0.176211,-2.746781,-3.983051,-9.504792,-12.644564,2.707973e+05,11.446945,0.116945,30.973202,38.420310,28.152327,31.565585,50.098731,63.610719
3542,ZXM,CI MORNINGSTAR INTL MOMENTUM IDX ET HEDGED COM...,35.360,1.902017,3.452311,5.269425,2.492754,-0.394366,1.304700e+03,34.335796,1.024204,65.557123,77.884344,64.654443,55.768688,73.455571,95.119887
3543,ZZZ,SLEEP COUNTRY CANADA HOLDINGS INC,25.350,2.053140,0.000000,10.940919,-2.537486,-17.292007,9.888503e+04,24.781762,0.568238,66.713681,62.496474,72.834979,40.028209,45.105783,86.770099


In [29]:
hqm_dataframe['Diff From Moving Avg Percentile'] = abs(hqm_dataframe['Diff From Moving Avg Percentile'] - 100)

In [30]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile
0,AAB,ABERDEEN INTERNATIONAL INC,0.040,-11.111111,-27.272727,-20.000000,-42.857143,-68.000000,8.963450e+04,0.046975,0.006975,7.489422,4.781382,12.440056,10.014104,10.592384,80.169252
1,AAG,AFTERMATH SILVER LTD,0.255,13.333333,-8.928571,-23.880597,50.000000,-32.000000,1.644119e+05,0.250693,0.004307,90.141044,23.413258,10.380818,91.283498,34.414669,85.613540
2,AALI,ADVANCE LITHIUM CORP,0.060,71.428571,140.000000,200.000000,300.000000,9.090909,4.760062e+05,0.033653,0.026347,99.181946,99.520451,98.970381,99.407616,85.923836,58.871650
3,AAU,ANGOLD RESOURCES LTD,0.025,25.000000,0.000000,-28.571429,-70.588235,-85.294118,3.024310e+05,0.024456,0.000544,95.345557,62.496474,8.011283,1.678420,3.018336,97.602257
4,AAV,ADVANTAGE ENERGY LTD,8.380,-2.558140,4.358655,-24.910394,-20.266413,10.993377,1.008012e+06,8.383976,0.003976,21.636107,79.407616,9.901269,23.159379,87.277856,86.629055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,ZWT,BMO COVERED CALL TECHNOLOGY ETF UNIT,26.610,3.179527,-0.708955,7.298387,6.567881,-10.404040,4.597122e+03,25.833127,0.776873,72.863188,54.442877,68.434415,64.598025,53.991537,9.929478
3541,ZWU,BMO ASSET MANAGEMENT INC COVERED CALL UTILITIE...,11.330,-0.176211,-2.746781,-3.983051,-9.504792,-12.644564,2.707973e+05,11.446945,0.116945,30.973202,38.420310,28.152327,31.565585,50.098731,36.389281
3542,ZXM,CI MORNINGSTAR INTL MOMENTUM IDX ET HEDGED COM...,35.360,1.902017,3.452311,5.269425,2.492754,-0.394366,1.304700e+03,34.335796,1.024204,65.557123,77.884344,64.654443,55.768688,73.455571,4.880113
3543,ZZZ,SLEEP COUNTRY CANADA HOLDINGS INC,25.350,2.053140,0.000000,10.940919,-2.537486,-17.292007,9.888503e+04,24.781762,0.568238,66.713681,62.496474,72.834979,40.028209,45.105783,13.229901


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [31]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [32]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile,HQM Score
0,AAB,ABERDEEN INTERNATIONAL INC,0.040,-11.111111,-27.272727,-20.000000,-42.857143,-68.000000,8.963450e+04,0.046975,0.006975,7.489422,4.781382,12.440056,10.014104,10.592384,80.169252,20.914433
1,AAG,AFTERMATH SILVER LTD,0.255,13.333333,-8.928571,-23.880597,50.000000,-32.000000,1.644119e+05,0.250693,0.004307,90.141044,23.413258,10.380818,91.283498,34.414669,85.613540,55.874471
2,AALI,ADVANCE LITHIUM CORP,0.060,71.428571,140.000000,200.000000,300.000000,9.090909,4.760062e+05,0.033653,0.026347,99.181946,99.520451,98.970381,99.407616,85.923836,58.871650,90.312647
3,AAU,ANGOLD RESOURCES LTD,0.025,25.000000,0.000000,-28.571429,-70.588235,-85.294118,3.024310e+05,0.024456,0.000544,95.345557,62.496474,8.011283,1.678420,3.018336,97.602257,44.692055
4,AAV,ADVANTAGE ENERGY LTD,8.380,-2.558140,4.358655,-24.910394,-20.266413,10.993377,1.008012e+06,8.383976,0.003976,21.636107,79.407616,9.901269,23.159379,87.277856,86.629055,51.335214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,ZWT,BMO COVERED CALL TECHNOLOGY ETF UNIT,26.610,3.179527,-0.708955,7.298387,6.567881,-10.404040,4.597122e+03,25.833127,0.776873,72.863188,54.442877,68.434415,64.598025,53.991537,9.929478,54.043253
3541,ZWU,BMO ASSET MANAGEMENT INC COVERED CALL UTILITIE...,11.330,-0.176211,-2.746781,-3.983051,-9.504792,-12.644564,2.707973e+05,11.446945,0.116945,30.973202,38.420310,28.152327,31.565585,50.098731,36.389281,35.933239
3542,ZXM,CI MORNINGSTAR INTL MOMENTUM IDX ET HEDGED COM...,35.360,1.902017,3.452311,5.269425,2.492754,-0.394366,1.304700e+03,34.335796,1.024204,65.557123,77.884344,64.654443,55.768688,73.455571,4.880113,57.033380
3543,ZZZ,SLEEP COUNTRY CANADA HOLDINGS INC,25.350,2.053140,0.000000,10.940919,-2.537486,-17.292007,9.888503e+04,24.781762,0.568238,66.713681,62.496474,72.834979,40.028209,45.105783,13.229901,50.068171


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [33]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
# hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [34]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[['Symbol',
    'Name',
    'Price',
    'HQM Score',
    'AvgVolume',
    'Exp Moving Average',
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'weeklyPerformance Percentile',
    '1monthPerformace Percentile',
    '3monthPerformace Percentile',
    '6monthPerformace Percentile',
    '1yearPerformace Percentile'
]]

In [36]:
hqm_dataframe[:50]

,Symbol,Name,Price,HQM Score,AvgVolume,Exp Moving Average,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile
0,CEQ,CRITERIUM ENERGY LTD,0.080,91.459803,276922.233333,0.072397,6.666667,60.000000,77.777778,100.000000,166.666667,82.172073,97.700987,94.710860,96.403385,98.815233
1,SNA,STAR NAVIGATION SYSTEMS GROUP LTD.,0.045,90.944993,448351.066667,0.039681,12.500000,12.500000,50.000000,125.000000,80.000000,89.393512,87.475317,90.677010,97.729196,97.179126
2,ITKO,IDAHO CHAMPION GOLD MINES CANADA INC.,0.150,90.620592,333795.844444,0.104864,50.000000,66.666667,200.000000,200.000000,130.769231,98.406206,98.025388,98.970381,98.928068,98.335684
3,SXL,SLAM EXPLORATION LTD,0.045,90.566526,96120.222222,0.031433,80.000000,50.000000,80.000000,80.000000,12.500000,99.322990,96.826516,94.894217,95.049365,88.025388
4,MDMA,PHARMALA BIOTECH HOLDINGS INC.,0.135,90.533615,846322.966667,0.084072,35.000000,125.000000,440.000000,200.000000,170.000000,97.179126,99.379408,99.774330,98.928068,98.857546
5,EOX,EUROMAX RESOURCES LTD,0.100,90.434885,45577.977778,0.080270,25.000000,42.857143,122.222222,122.222222,33.333333,95.345557,95.980254,97.306065,97.545839,92.877292
6,MUSH,THE GOOD SHROOM CO INC,0.135,90.430183,83692.222222,0.103673,28.571429,58.823529,350.000000,170.000000,42.105263,96.220028,97.574048,99.633286,98.589563,94.372355
7,ERA,ELCORA ADVANCED MATERIALS CORP,0.105,90.319699,153061.333333,0.055285,90.909091,133.333333,162.500000,133.333333,75.000000,99.492243,99.464034,98.660085,97.856135,96.939351
8,AALI,ADVANCE LITHIUM CORP,0.060,90.312647,476006.222222,0.033653,71.428571,140.000000,200.000000,300.000000,9.090909,99.181946,99.520451,98.970381,99.407616,85.923836
9,NED,NEW DESTINY MINING CORPORATION,0.100,90.197461,54980.177778,0.054167,42.857143,42.857143,400.000000,233.333333,100.000000,97.785614,95.980254,99.689704,99.181946,97.799718


In [37]:
high_volume_df = hqm_dataframe[hqm_dataframe['AvgVolume'] > 1000000][:50]

In [38]:
high_volume_df.reset_index(inplace=True, drop=True)

In [39]:
high_volume_df

,Symbol,Name,Price,HQM Score,AvgVolume,Exp Moving Average,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile
0,SBB,SABINA GOLD & SILVER CORPORATION,1.8200,80.267983,1.265598e+06,1.623031,6.432749,34.814815,45.600000,82.000000,28.169014,81.889986,94.753173,89.788434,95.232722,91.932299
1,PEI,PROSPERA ENERGY INC,0.1350,80.082276,1.057892e+06,0.132626,-3.571429,8.000000,58.823529,145.454545,58.823529,18.970381,84.160790,92.143865,98.180536,96.121298
2,ATH,ATHABASCA OIL CORP,3.2900,77.465914,4.462621e+06,2.844179,11.148649,18.345324,28.015564,40.598291,66.161616,88.067701,90.803949,83.836389,89.308886,96.445698
3,FUU,F3 URANIUM CORP,0.4200,75.742830,2.478348e+06,0.391776,-3.448276,25.373134,35.483871,281.818182,147.058824,19.308886,93.032440,86.685472,99.322990,98.504937
4,OGC,OCEANAGOLD CORPORATION,2.7500,74.268923,1.322063e+06,2.707978,4.961832,0.732601,9.126984,52.777778,4.166667,78.702398,70.070522,71.114245,91.791255,81.974612
5,AOI,AFRICA OIL CORP,3.0000,72.764457,1.045718e+06,2.801314,4.529617,10.701107,9.090909,20.000000,32.158590,77.517630,86.205924,70.973202,81.354020,92.609309
6,MEG,MEG ENERGY CORP,23.5200,72.637518,2.377792e+06,21.750452,7.152620,11.416390,24.774536,39.089296,33.560477,83.131171,86.770099,82.002821,88.829337,93.004231
7,CS,CAPSTONE COPPER CORP,6.6200,72.190879,1.515964e+06,6.103303,17.375887,7.467532,39.957717,124.406780,-8.563536,92.693935,83.497884,87.898449,97.686883,57.066291
8,CMMC,COPPER MOUNTAIN MINING CORPORATION,2.3000,68.707099,1.407438e+06,2.153305,13.861386,9.523810,25.000000,57.534247,-39.473684,90.409027,85.317348,82.327221,92.581100,28.970381
9,WCP,WHITECAP RESOURCES INC,10.8800,68.610719,2.452414e+06,10.648690,3.816794,4.414587,3.717827,21.158129,12.396694,75.585331,79.464034,60.592384,82.228491,87.926657


In [40]:
top_15 = high_volume_df[:15]

### Getting Historical data

In [41]:
from yahoo_fin.stock_info import get_data
from datetime import date, timedelta
import time

In [42]:
top_15_symbols = list(top_15['Symbol'].values)
top_15_history_data = {}

end_date_today = date.today().strftime("%d/%m/%Y")
start_date_90days_before = (date.today() - timedelta(days=130)).strftime("%m/%d/%Y")

for symbol in top_15_symbols:
    top_15_history_data[symbol] = get_data(symbol, start_date = start_date_90days_before, end_date = end_date_today, interval = '1d')
    print(symbol)
    time.sleep(1)

SBB
PEI
ATH
FUU
OGC
AOI
MEG
CS
CMMC
WCP
SU
MFC
NILI
EFN
CNQ


In [44]:
# top_15_history_data['TAL']

In [45]:
for symbol in top_15_history_data.keys():
    top_15_history_data[symbol]['open to low diff'] = top_15_history_data[symbol]['open'] - top_15_history_data[symbol]['low']
    top_15_history_data[symbol]['open to high diff'] = top_15_history_data[symbol]['high'] - top_15_history_data[symbol]['open']
    top_15_history_data[symbol]['low to high diff'] = top_15_history_data[symbol]['high'] - top_15_history_data[symbol]['low']
    

In [46]:
for row in top_15.index:
    top_15.loc[row, 'Avg open to low diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['open to low diff'].mean()
    top_15.loc[row, 'Avg open to high diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['open to high diff'].mean()
    top_15.loc[row, 'Avg low to high diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['low to high diff'].mean()
    

/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [47]:
# top_15_history_data['TAL']['open to high diff'].mean()

In [48]:
list(top_15.columns.values)

['Symbol',
 'Name',
 'Price',
 'HQM Score',
 'AvgVolume',
 'Exp Moving Average',
 'weeklyPerformance',
 '1monthPerformace',
 '3monthPerformace',
 '6monthPerformace',
 '1yearPerformace',
 'weeklyPerformance Percentile',
 '1monthPerformace Percentile',
 '3monthPerformace Percentile',
 '6monthPerformace Percentile',
 '1yearPerformace Percentile',
 'Avg open to low diff',
 'Avg open to high diff',
 'Avg low to high diff']

In [49]:
top_15 = top_15[[
    'Symbol',
    'Name',
    'Price',
    'HQM Score',
    'AvgVolume',
    'Exp Moving Average',
    'Avg open to low diff',
    'Avg open to high diff',
    'Avg low to high diff',
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'weeklyPerformance Percentile',
    '1monthPerformace Percentile',
    '3monthPerformace Percentile',
    '6monthPerformace Percentile',
    '1yearPerformace Percentile',
]]

In [50]:
top_15

,Symbol,Name,Price,HQM Score,AvgVolume,Exp Moving Average,Avg open to low diff,Avg open to high diff,Avg low to high diff,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile
0,SBB,SABINA GOLD & SILVER CORPORATION,1.820,80.267983,1.265598e+06,1.623031,0.232955,0.292727,0.525682,6.432749,34.814815,45.600000,82.000000,28.169014,81.889986,94.753173,89.788434,95.232722,91.932299
1,PEI,PROSPERA ENERGY INC,0.135,80.082276,1.057892e+06,0.132626,0.232955,0.292727,0.525682,-3.571429,8.000000,58.823529,145.454545,58.823529,18.970381,84.160790,92.143865,98.180536,96.121298
2,ATH,ATHABASCA OIL CORP,3.290,77.465914,4.462621e+06,2.844179,0.232955,0.292727,0.525682,11.148649,18.345324,28.015564,40.598291,66.161616,88.067701,90.803949,83.836389,89.308886,96.445698
3,FUU,F3 URANIUM CORP,0.420,75.742830,2.478348e+06,0.391776,0.232955,0.292727,0.525682,-3.448276,25.373134,35.483871,281.818182,147.058824,19.308886,93.032440,86.685472,99.322990,98.504937
4,OGC,OCEANAGOLD CORPORATION,2.750,74.268923,1.322063e+06,2.707978,0.232955,0.292727,0.525682,4.961832,0.732601,9.126984,52.777778,4.166667,78.702398,70.070522,71.114245,91.791255,81.974612
5,AOI,AFRICA OIL CORP,3.000,72.764457,1.045718e+06,2.801314,0.232955,0.292727,0.525682,4.529617,10.701107,9.090909,20.000000,32.158590,77.517630,86.205924,70.973202,81.354020,92.609309
6,MEG,MEG ENERGY CORP,23.520,72.637518,2.377792e+06,21.750452,0.232955,0.292727,0.525682,7.152620,11.416390,24.774536,39.089296,33.560477,83.131171,86.770099,82.002821,88.829337,93.004231
7,CS,CAPSTONE COPPER CORP,6.620,72.190879,1.515964e+06,6.103303,0.232955,0.292727,0.525682,17.375887,7.467532,39.957717,124.406780,-8.563536,92.693935,83.497884,87.898449,97.686883,57.066291
8,CMMC,COPPER MOUNTAIN MINING CORPORATION,2.300,68.707099,1.407438e+06,2.153305,0.232955,0.292727,0.525682,13.861386,9.523810,25.000000,57.534247,-39.473684,90.409027,85.317348,82.327221,92.581100,28.970381
9,WCP,WHITECAP RESOURCES INC,10.880,68.610719,2.452414e+06,10.648690,0.232955,0.292727,0.525682,3.816794,4.414587,3.717827,21.158129,12.396694,75.585331,79.464034,60.592384,82.228491,87.926657
